## Sandbox

Un  `.ipynb` pour tester des idées :)

In [ ]:
using Pkg
Pkg.activate(".")
Pkg.instantiate()
using Graphs
using MetaGraphsNext
using JuMP
using Gurobi

ENV["GRB_LICENSE_FILE"] = "gurobi.lic"
include(joinpath("src", "utils.jl"));
include(joinpath("src", "dual_solve.jl"));
include(joinpath("src", "testing.jl"));

In [ ]:
dual = completeModelWrapper(dualSolve, time_budget=300.0)

In [ ]:
save_file = joinpath("data", "results", "test.json")
instance_dir = joinpath("data", "small_instances")
testMethod(
    method=dual;
    method_name="Dualisation",
    save=save_file,
    instance_dir=instance_dir,
)